------------

> **❗❗❗⚠️  💸   USE A T4 GPU or above❗❗❗**   

------------

# Testing FLOR-1.3B Customer Service finetuned

In this Notebook, we load and test the "avalosjc/flor-1-3B-customerservice-es" model, a fine-tuned version of FLOR-1.3B. Hosted on Hugging Face, this model is specialized in customer service queries and answers. Our aim is to verify its functionality and assess its ability to handle specific inquiries in the customer service domain.


In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 60.3 MB/s eta 0:00:00
ERROR: pip's 

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id  = "avalosjc/flor-1-3B-customerservice-es"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/844k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/503k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

In [5]:
system_message = "Below is an instruction that describes a task. You are a helpfull customer service assistant. Answer always in spanish."

In [6]:
def prompt_template(input, response):

  full_prompt = "<|startoftext|>"
  full_prompt += "\n### Instruction\n"
  full_prompt += system_message + "\n\n"
  full_prompt += "### Context\n"
  full_prompt += input + "\n\n"
  full_prompt += "### Answer\n"
  if response != "":
    full_prompt += response
    full_prompt += "\n<|endoftext|>"

  return full_prompt

def create_prompt(sample):
  input = sample["instruction"]
  response = sample["response"]

  full_prompt = prompt_template(input, response)

  return full_prompt

In [7]:
def generate_response(prompt, model, tokenizer):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=60, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [8]:
response = generate_response(prompt_template("Como acceder a un factura electrónica?", ""),
                  model,
                  tokenizer)
print(response)

Lo sentimos, nuestro sistema no procesa facturas electrónicas. Para acceder a una factura electrónica, debe proporcionarnos los detalles de su factura actual, incluido el número de factura y su país de facturación, o puede comunicarse con nuestro equipo de atención al cliente para obtener más información.
<|endoftext|>


In [9]:
response = generate_response(prompt_template("Como debo proceder para devolver un producto?", ""),
                  model,
                  tokenizer)
print(response)

Primeramente, le agradecemos que se comunique con nosotros para hacernos esta pregunta. Permítame responderle a continuación.

El reembolso de su reembolso debe ser presentado en nuestra oficina dentro de los 8 días hábiles de la entrega del artículo. Le enviaremos una Nota de Reembolso (


In [10]:
response = generate_response(prompt_template("Quien es Javier Milei?", ""),
                  model,
                  tokenizer)
print(response)

Soy un argentino de Buenos Aires de 52 años. Me licencié en la universidad por la carrera de Ciencias Contables en la Universidad Nacional de Rosario. Posteriormente, trabajé en una empresa financiera durante dos años. Después de un año de búsqueda y entrenamiento, me convertí en vendedor dedicado de ventas por


In [11]:
response = generate_response(prompt_template("Donde queda Argentina?", ""),
                  model,
                  tokenizer)
print(response)

Argentina, commonly referred to as "Argentina", is a modern country with an impressive history, and is commonly associated with tango, a national anthem. Argentina is also the native country of Argentinean footballer Lionel Messi, the most successful player of all time. There are also a


In [12]:
response = generate_response(prompt_template("Como acceder a un factura electrónica?", ""),
                  model,
                  tokenizer)
print(response)

Por favor, póngase en contacto con nuestro departamento de facturación para obtener ayuda sobre dónde y cómo acceder a su factura electrónica. Agradecemos su paciencia en espera de nuestra respuesta.
<|endoftext|>


## Resultado

A veces da respuestas que NO debería.

Lo más raro es que responda en inglés cuando se le consultó "Donde queda Argentina?